**MCMC(j)- TCI(k,l)**

In [3]:
import TensorCrossInterpolation as TCI




function f(v)
    j = v[1]
    k = v[2]
    l = v[3]
    
    NSq = N2^2
    
    term1 = cos(2*π*(j*k/NSq)) * sin(2*π*(j*l/NSq)) * exp(-j/N)
    term2 = sin(2*π*(j*k/NSq)) * cos(2*π*(j*l/NSq)) * exp(-j^2/NSq)

    return term1 + term2
end


function get_tci_val(j)
    localdims = fill(N2, 2)
    tolerance = 1e-8

    g(v) = f([j, v[1], v[2]]) #the method of fix one variable
    
    tci_2d, _, _ = TCI.crossinterpolate2(Float64, g, localdims; tolerance = tolerance )
    
    mps = tci_2d.sitetensors
    
    sumK = dropdims(sum(mps[1],dims=2),dims=2) #(r1 X N X r2) -> (r1 X 1 X r2) -> (r1 x r2)
    sumL = dropdims(sum(mps[2],dims=2),dims=2)

    resultMatrix = sumK * sumL #1x1 matrix
    
    return resultMatrix[1]

end


function mcmc_tci_sum(M)
    #define sum value
    sumVal = 0.0

    #set init point
    j = rand(1:N)
    now = get_tci_val(j)
    memJ[j] = now
    sumVal += now
    

    for idx in 1:(M-1)
        jump = rand(-round(Int,N/100):round(Int,N/100))
        
        if (j + jump) < 1 || (j + jump) > N
            sumVal += now
            continue
        else
            j = j + jump

            #calculate proposal point
            prp = 0.0
            if memJ[j] == 0
                prp = get_tci_val(j)
                memJ[j] = prp
            else
                prp = memJ[j]
            end

            #MCMC
            accRatio = prp/now
            if rand() < accRatio
                #acc
                sumVal += prp
                now = prp
            else
                #rej
                sumVal += now 
                j = j - jump #undo
            end
        end
    end

    return sumVal
end

function exact_sum()
    sum = 0.0
    
    for j in 1:N
        for k in 1:N2
            for l in 1:N2
                sum += f([j, k, l])
            end
        end
    end
    return sum / (N*(N2^2))
end

N = 10_000 #Num of j elements
N2 = 100 #Num of k,l elements
MLst = [10,100,1_000] #number of sampling
memJ = zeros(N)


extVal = exact_sum()
println("exact result : $extVal")

 for M in MLst
    println("====M : $M====")
    epoch = 10
    acc = 0
    accSq = 0
    @time for i in 1:epoch

        mcmcTciSum = mcmc_tci_sum(M) / (M*(N2^2))

        acc += mcmcTciSum
        accSq += mcmcTciSum^2
    end

    stdev = sqrt((accSq / epoch - (acc / epoch)^2)/epoch)

    println("MCMC-TCI result : $(acc/epoch)")
    println("stdev/sqrt(M) : $(stdev/sqrt(M))")

    if extVal - (stdev/sqrt(M)) <  (acc/epoch) && (acc/epoch) < extVal + (stdev/sqrt(M))
        println("Success :)")
    else
        println("Fail :(")
    end

end

exact result : 0.0017325109251710056
====M : 10====
  6.974665 seconds (173.10 M allocations: 3.098 GiB, 6.42% gc time, 19.97% compilation time)
MCMC-TCI result : 0.0019806641688804643
stdev/sqrt(M) : 0.0005924514396763556
Success :)
====M : 100====
 49.013862 seconds (1.50 G allocations: 26.405 GiB, 8.21% gc time)
MCMC-TCI result : 0.0009262130380829536
stdev/sqrt(M) : 8.508583448990623e-5
Fail :(
====M : 1000====
234.495688 seconds (7.26 G allocations: 127.453 GiB, 9.20% gc time)
MCMC-TCI result : -0.025575409333582676
stdev/sqrt(M) : 0.00046248825025505626
Fail :(


**Proposal Distribution**
https://jduarte.physics.ucsd.edu/phys142/lectures/16_Path_Integral_MCMC.pdf (page 9)\
There's no need to use complicated proposal distribution. \
We can simply use vuniform distribution.